In [ ]:
import Pkg

In [ ]:
Pkg.activate(".")

In [ ]:
Pkg.update()

In [ ]:
import QuantumGrav

In [ ]:
import Plots
import LinearAlgebra
import Random
import Distributions 
import SparseArrays
import JLD2
import ProgressMeter

In [ ]:
rng = Random.Xoshiro(12433)

# Make data

In [ ]:
rngs = [Random.Xoshiro(12433 + i) for i in 1:Threads.nthreads()]

println(Threads.nthreads())

csets = [[] for _ in 1:Threads.nthreads()]
conv = [[] for _ in 1:Threads.nthreads()]
n = 10000
s = 512
p = ProgressMeter.Progress(n, 1, "Generating causets...")

Threads.@threads for i in 1:n
    t = Threads.threadid()
    rng = rngs[t]

    converged = false 
    while converged == false 
        cset, convg = QuantumGrav.sample_bitarray_causet_by_connectivity(
            s,
            0.5,
            30,
            rng;
            abs_tol = 1e-3,
            acceptance = 5e5,
        )

        converged = convg

        if converged
            push!(csets[t], cset)
        end

        push!(conv[t], convg)
    end
    ProgressMeter.next!(p)
end
ProgressMeter.finish!(p)

csets = vcat(csets...)

convs = vcat(conv...)


JLD2.@save "random_csets.jld2" csets convs

In [ ]:
csets_manifold = [[] for _ in 1:Threads.nthreads()]

p = ProgressMeter.Progress(n, 1, "Generating adjs...")
Threads.@threads for i in 1:n
    t = Threads.threadid()
    rng = rngs[t]

    cset, _, __ = QuantumGrav.make_manifold_cset(
        s,
        rng,
        24,
        3.0,
    )
    push!(csets_manifold[t], cset)
    ProgressMeter.next!(p)
end
ProgressMeter.finish!(p)

JLD2.@save "manifold_csets.jld2" csets_manifold


In [ ]:
JLD2.@load "manifold_csets.jld2" csets_manifold
JLD2.@load "random_csets.jld2" csets convs
csets_random = csets 

In [ ]:
numbroken = count(x -> x == false, convs)
println("Number of causets: ", length(csets))
println("Number of broken causets: ", numbroken)
println("Fraction of broken causets: ", numbroken / length(csets))


## Make adj matrices

In [ ]:
to_adj(cset) = reduce(hcat, SparseArrays.sparse.(cset.future_relations)) |> transpose |> SparseArrays.sparse

In [ ]:

adjs = [[] for _ in 1:Threads.nthreads()]
p = ProgressMeter.Progress(n, 1, "Generating adjs...")
Threads.@threads for cset in csets
    t = Threads.threadid()
    push!(adjs[t], to_adj(cset))
    ProgressMeter.next!(p)
end
ProgressMeter.finish!(p)

adjs = vcat(adjs...)
JLD2.@save "random_adjs.jld2" adjs

In [ ]:
adjs = [[] for _ in 1:Threads.nthreads()]
p = ProgressMeter.Progress(n, 1, "Generating adjs...")
Threads.@threads for cset in csets_manifold
    t = Threads.threadid()
    push!(adjs[t], to_adj(cset))
    ProgressMeter.next!(p)
end
ProgressMeter.finish!(p)

adjs = vcat(adjs...)
JLD2.@save "manifold_adjs.jld2" adjs


In [ ]:
JLD2.@load "manifold_adjs.jld2" adjs
manifold_adjs = deepcopy(adjs)
JLD2.@load "random_adjs.jld2" adjs
random_adjs = deepcopy(adjs)

## Make Degree Matrices

In [ ]:
degs(adj) = LinearAlgebra.Diagonal(sum(Int.(adj), dims=2)[:, 1]) |> SparseArrays.sparse


In [ ]:

future_degrees = [[] for _ in 1:Threads.nthreads()]
p = ProgressMeter.Progress(n, 1, "Calculating future degrees...")
Threads.@threads for adj in adjs
    t = Threads.threadid()
    push!(future_degrees[t], degs(adj))
    ProgressMeter.next!(p)
end
ProgressMeter.finish!(p)
future_degrees = vcat(future_degrees...)
JLD2.@save "random_degs.jld2" future_degrees


In [ ]:

future_degrees = [[] for _ in 1:Threads.nthreads()]
p = ProgressMeter.Progress(n, 1, "Calculating future degrees...")
Threads.@threads for adj in adjs
    t = Threads.threadid()
    push!(future_degrees[t], degs(adj))
    ProgressMeter.next!(p)
end
ProgressMeter.finish!(p)
future_degrees = vcat(future_degrees...)
JLD2.@save "manifold_degs.jld2" future_degrees


In [ ]:
JLD2.@load "manifold_degs.jld2" future_degrees
manifold_degs = deepcopy(future_degrees)

JLD2.@load "random_degs.jld2" future_degrees
random_degs = deepcopy(future_degrees)

future_degrees = nothing 

# Make Laplacian

In [ ]:
lap(adj) = degs(adj) - adj


In [ ]:

future_laps = [[] for _ in 1:Threads.nthreads()]
p = ProgressMeter.Progress(n, 1, "Calculating future laplacians...")
Threads.@threads for adj in adjs
    t = Threads.threadid()
    push!(future_laps[t], lap(adj))
    ProgressMeter.next!(p)
end
ProgressMeter.finish!(p)

future_laps = vcat(future_laps...)
JLD2.@save "random_laps.jld2" future_laps



In [ ]:
future_laps = [[] for _ in 1:Threads.nthreads()]
p = ProgressMeter.Progress(n, 1, "Calculating future laplacians...")
Threads.@threads for adj in adjs
    t = Threads.threadid()
    push!(future_laps[t], degs(adj) - adj)
    ProgressMeter.next!(p)
end
ProgressMeter.finish!(p)

future_laps = vcat(future_laps...)
JLD2.@save "manifold_laps.jld2" future_laps


In [ ]:
JLD2.@load "manifold_laps.jld2" future_laps
manifold_laps = deepcopy(future_laps)

JLD2.@load "random_laps.jld2" future_laps
random_laps = deepcopy(future_laps)

future_laps = nothing

# Max pathlens

In [ ]:
n = size(manifold_adjs[1], 1)

## find sources and sinks

A source is a node that has no predessesors <=> it's past relations are empty <=> it's past degree is zero. likewise for sinks, the same holds but into the future

In [ ]:
csets_manifold = vcat(csets_manifold...)

In [ ]:
manifold_sources = []
manifold_sinks = []
for cset in csets_manifold
    source_nodes = []
    sink_nodes = []
    for (i,v) in enumerate(cset.past_relations) 
        if all(v .== false)
            push!(source_nodes, i)
        end
    end

    for (i,v) in enumerate(cset.future_relations)
        if all(v .== false)
            push!(sink_nodes, i)
        end
    end

    push!(manifold_sources, source_nodes)
    push!(manifold_sinks, sink_nodes)
end

random_sources = []
random_sinks = []
for cset in csets_random
    source_nodes = []
    sink_nodes = []
    for (i,v) in enumerate(cset.past_relations)
        if all(v .== false)
            push!(source_nodes, i)
        end
    end

    for (i,v) in enumerate(cset.future_relations)
        if all(v .== false)
            push!(sink_nodes, i)
        end
    end

    push!(random_sources, source_nodes)
    push!(random_sinks, sink_nodes)
end

In [ ]:
random_source_numbers = [length(s) for s in random_sources]


In [ ]:
manifold_source_numbers = [length(s) for s in manifold_sources]

In [ ]:
JLD2.@save "manifold_sources.jld2" manifold_sources
JLD2.@save "manifold_sinks.jld2" manifold_sinks
JLD2.@save "random_sources.jld2" random_sources
JLD2.@save "random_sinks.jld2" random_sinks

In [ ]:
pathlens_manifold = [[] for _ in 1:Threads.nthreads()]
Threads.@threads for i in 1:length(manifold_adjs)
    adj = manifold_adjs[i]
    pathlens_i = [QuantumGrav.max_pathlen(adj, collect(1:n), s)
                  for s in manifold_sources[i]]
    push!(pathlens_manifold[Threads.threadid()], pathlens_i)
end  
pathlens_manifold = vcat(pathlens_manifold...)

pathlens_random = [[] for _ in 1:Threads.nthreads()]
Threads.@threads for i in 1:length(random_adjs)
    adj = random_adjs[i]
    pathlens_i = [QuantumGrav.max_pathlen(adj, collect(1:n), s)
                  for s in random_sources[i]]
    push!(pathlens_random[Threads.threadid()], pathlens_i)
end
pathlens_random = vcat(pathlens_random...)

JLD2.@save "manifold_pathlens.jld2" pathlens_manifold
JLD2.@save "random_pathlens.jld2" pathlens_random

## Laplacian EVs

In [ ]:
JLD2.@load "manifold_laps.jld2" future_laps
manifold_laps = deepcopy(future_laps)

JLD2.@load "random_laps.jld2" future_laps
random_laps = deepcopy(future_laps)

future_laps = nothing

In [ ]:
num_zeros = []
five_largest = []
for i in 1:length(manifold_laps)
    m = LinearAlgebra.eigen(Matrix{Int64}(manifold_laps[i]))
    nz = count(x -> abs(x)< 1e-9, m.values)
    fv = m.values[end-5:end]
    push!(num_zeros, nz)
    push!(five_largest, fv)
end

JLD2.@save "manifold_lap_num_zero_ev.jld2" num_zeros
JLD2.@save "manifold_lap_five_largest_ev.jld2" five_largest

In [ ]:
num_zeros = []
five_largest = []
for i in 1:length(random_laps)
    m = LinearAlgebra.eigen(Matrix{Int64}(random_laps[i]))
    nz = count(x -> abs(x)< 1e-9, m.values)
    fv = m.values[end-5:end]
    push!(num_zeros, nz)
    push!(five_largest, fv)
end

JLD2.@save "random_lap_num_zero_ev.jld2" num_zeros
JLD2.@save "random_lap_five_largest_ev.jld2" five_largest

# Plot data

In [ ]:
using Plots
using JLD2 
using Statistics

## Degree distributions

In [ ]:
JLD2.@load "manifold_degs.jld2" future_degrees
manifold_degs = deepcopy(future_degrees)
JLD2.@load "random_degs.jld2" future_degrees
random_degs = deepcopy(future_degrees)

In [ ]:
diag_vecs_manifold = map(x->LinearAlgebra.diag(x)|> Vector{Int64}, manifold_degs)

In [ ]:
diag_vecs_random = map(x->LinearAlgebra.diag(x)|> Vector{Int64}, random_degs)

In [ ]:
min_degree_manifold = minimum.(diag_vecs_manifold)
max_degree_manifold = maximum.(diag_vecs_manifold)
median_degree_manifold = median.(diag_vecs_manifold)   
q25_degree_manifold = quantile.(diag_vecs_manifold, 0.25)
q75_degree_manifold = quantile.(diag_vecs_manifold, 0.75)

In [ ]:
min_degree_random = minimum.(diag_vecs_random)
max_degree_random = maximum.(diag_vecs_random)
median_degree_random = median.(diag_vecs_random)   
q25_degree_random = quantile.(diag_vecs_random, 0.25)
q75_degree_random = quantile.(diag_vecs_random, 0.75)

In [ ]:
p1 = histogram(bins=-0.5:5:500.5, min_degree_manifold, title="Min degree Manifold", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)

p2 = histogram(bins=-0.5:5:500.5, q25_degree_manifold, title="Q25 degree Manifold", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)

p3 = histogram(bins=-0.5:5:500.5, median_degree_manifold, title="Median degree Manifold", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)

p4 = histogram(bins=-0.5:5:500.5,q75_degree_manifold, title="Q75 degree Manifold", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)

p5 = histogram(bins=-0.5:5:500.5,max_degree_manifold, title="Max degree Manifold", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)

In [ ]:
histogram!(p1, bins=-0.5:5:500.5, min_degree_random, title="Min future degree random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

histogram!(p2, bins=-0.5:5:500.5, q25_degree_random, title="Q25 future degree random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

histogram!(p3, bins=-0.5:5:500.5, median_degree_random, title="Median future degree random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

histogram!(p4, bins=-0.5:5:500.5,q75_degree_random, title="Q75 future degree random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

histogram!(p5, bins=-0.5:5:500.5,max_degree_random, title="Max future degree random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

In [ ]:
p = plot(p1, p2, p3, p4, p5, layout=(3, 2), size=(800, 600), alpha=0.7)


In [ ]:
savefig(p, "degree_histograms.png")

## number of sinks and sources 

In [ ]:
JLD2.@load "manifold_sources.jld2" manifold_sources
JLD2.@load "manifold_sinks.jld2" manifold_sinks
JLD2.@load "random_sources.jld2" random_sources
JLD2.@load "random_sinks.jld2" random_sinks

In [ ]:
num_manifold_sources = [length(s) for s in manifold_sources]
num_random_sources = [length(s) for s in random_sources]

num_manifold_sinks = [length(s) for s in manifold_sinks]
num_random_sinks = [length(s) for s in random_sinks]

In [ ]:
p1 = histogram(-0.5:100.5, num_random_sources, xlabel="Value", title=" Sources", ylabel="Frequency", label="random", alpha = 0.7);

p2 = histogram(bins=-0.5:12.5, num_random_sinks, xlabel="Value", title=" Sinks", ylabel="Frequency", label="random", alpha = 0.7);

histogram!(p1, -0.5:100.5, num_manifold_sources, label="manifolds", xlabel="Value", ylabel="Frequency", alpha = 0.7);

histogram!(p2, bins=-0.5:12.5, num_manifold_sinks, xlabel="Value", ylabel="Frequency", label="manifolds", alpha = 0.7);

In [ ]:
p = plot(p1, p2, layout=(2,1), size=(800, 600),)

In [ ]:
savefig(p, "sink_sources_histogram.png")

## max pathlens

In [ ]:
using Statistics

In [ ]:
JLD2.@load "manifold_pathlens.jld2" pathlens_manifold
JLD2.@load "random_pathlens.jld2" pathlens_random

In [ ]:
min_pathlens_manifold = minimum.(pathlens_manifold)
max_pathlens_manifold = maximum.(pathlens_manifold)
median_pathlens_manifold = median.(pathlens_manifold)   
q25_pathlens_manifold = quantile.(pathlens_manifold, 0.25)
q75_pathlens_manifold = quantile.(pathlens_manifold, 0.75)

In [ ]:
p1 = histogram(bins =-0.5:75.5, min_pathlens_manifold, title="Min Pathlens ", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)

p2 = histogram(bins =-0.5:75.5, q25_pathlens_manifold, title="Q25 Pathlens ", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)

p3 = histogram(bins =-0.5:75.5, median_pathlens_manifold, title="Median Pathlens ", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)

p4 = histogram(bins =-0.5:75.5, q75_pathlens_manifold, title="Q75 Pathlens ", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)

p5 = histogram(bins =-0.5:75.5, max_pathlens_manifold, title="Max Pathlens ", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)


In [ ]:
min_pathlens_random = minimum.(pathlens_random)
max_pathlens_random = maximum.(pathlens_random)
median_pathlens_random = median.(pathlens_random)   
q25_pathlens_random = quantile.(pathlens_random, 0.25)
q75_pathlens_random= quantile.(pathlens_random, 0.75)

In [ ]:
histogram!(p1, bins =-0.5:75.5, min_pathlens_random, title="Min Pathlens Manifold", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

histogram!(p2, bins =-0.5:75.5, q25_pathlens_random, title="Q25 Pathlens Manifold", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

histogram!(p3, bins =-0.5:75.5, median_pathlens_random, title="Median Pathlens Manifold", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

histogram!(p4, bins =-0.5:75.5, q75_pathlens_random, title="Q75 Pathlens Manifold", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

histogram!(p5, bins =-0.5:75.5, max_pathlens_random, title="Max Pathlens Manifold", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

In [ ]:
p = plot(p1, p2, p3, p4, p5, layout=(3, 2), size=(800, 600),)

In [ ]:
savefig(p, "pathlens_histogram.png")

## number of zero EVs

In [ ]:
JLD2.@load "manifold_lap_num_zero_ev.jld2" num_zeros
num_zeros_manifold = deepcopy(num_zeros)

JLD2.@load "random_lap_num_zero_ev.jld2" num_zeros
num_zeros_random = deepcopy(num_zeros)

In [ ]:
p = histogram(bins= -0.5:100.5, num_zeros_manifold, title="Number of zero eigenvalues manifold", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)
histogram!(p, bins= -0.5:100.5, num_zeros_random, title="Number of zero eigenvalues random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

In [ ]:
savefig(p, "num_zero_evs_histogram.png")

## five largest evs

In [ ]:
JLD2.@load "manifold_lap_five_largest_ev.jld2" five_largest
five_largest_manifold = deepcopy(five_largest)

JLD2.@load "random_lap_five_largest_ev.jld2" five_largest
five_largest_random = deepcopy(five_largest)

In [ ]:
largest_evs_manifold = hcat(five_largest_manifold...)
largest_evs_random = hcat(five_largest_random...)

In [ ]:
p1 = histogram(bins=-0.5:5:500.5, largest_evs_manifold[1, :], title="1st ev", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)
histogram!(p1, bins=-0.5:5:500.5, largest_evs_random[1, :], title="1st ev random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

p2 = histogram(bins=-0.5:5:500.5, largest_evs_manifold[2, :], title="2nd ev", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)
histogram!(p2, bins=-0.5:5:500.5, largest_evs_random[2, :], title="2nd ev random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

p3 = histogram(bins=-0.5:5:500.5, largest_evs_manifold[3, :], title="3rd ev", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)
histogram!(p3, bins=-0.5:5:500.5, largest_evs_random[3, :], title="3rd ev random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

p4 = histogram(bins=-0.5:5:500.5, largest_evs_manifold[4, :], title="4th ev", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)
histogram!(p4, bins=-0.5:5:500.5, largest_evs_random[4, :], title="4th ev random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

p5 = histogram(bins=-0.5:5:500.5, largest_evs_manifold[5, :], title="5th ev", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)
histogram!(p5, bins=-0.5:5:500.5, largest_evs_random[5, :], title="5th ev random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

p6 = histogram(bins=-0.5:5:500.5, largest_evs_manifold[6, :], title="6th ev", xlabel="Value", ylabel="Frequency", label="manifold", alpha=0.7)
histogram!(p6, bins=-0.5:5:500.5, largest_evs_random[6, :], title="6th ev random", xlabel="Value", ylabel="Frequency", label="random", alpha=0.7)

p = plot(p1, p2, p3, p4, p5, p6, layout=(3, 2), size=(800, 600), alpha=0.7)

In [ ]:
savefig(p, "largest_evs_histogram.png")